In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib as mp
import re

In [2]:
def log(f):
    #f
    return
def wrn(f):
    #f
    return


In [3]:
def str_to_second(s, multiby):
    if s=="":
        return 0.0
    else:
        return float(s)*multiby

def normalize(ss):
#     log(print("ss:",ss, len(ss)))
#     if len(ss)==1:
#         res = ss
#         res.append('')
#         log(print("res:",res))
#         return res
#     else:
    return ss
    
def time_to_second(tstr):
    t = ""
    i = 0
    l = len(tstr)
    time = 0
    while i < l:
        if tstr[i]=='h':
            time += str_to_second(t,3600)   #hour
            t = ""
        elif tstr[i]== 'm':
            if i==l-1 or tstr[i+1] != 's':
                time += str_to_second(t,60)   #minute
            else:
                i += 1
                time +=  str_to_second(t,1.0/1000)  #ms
            t = ""
        elif tstr[i]== 's':
            time +=  str_to_second(t,1) #second
            t=""
        elif tstr[i]== 'µ':
            i += 1
            time +=  str_to_second(t,1.0/1000000)  #us
            t = ""
        elif tstr[i]=='Â':
            i += 2
            time +=  str_to_second(t,1.0/1000000)  #us
            t = ""
        elif tstr[i]== "n":
            i += 1
            time +=  str_to_second(t,1.0/1000000000)  #ns
            t = ""
        else:
            t += tstr[i]
                
        i += 1
    
    if time==0:
        time=1.0/10000000
    return time


In [4]:
def get_report(fn):
    with open(fn) as f:
        lines = [line.rstrip('\n') for line in f  if line.startswith("Testing started") or line.startswith("Writing entries") or line.startswith("Searching time")]
        
    if len(lines)!=3:
        #wrn(print("Invalid input with ",len(lines)," lines in ",fn))
        return [0,0,0],False
    
    wt = time_to_second(lines[1][lines[1].find("time:  ")+len("time:  "):])
    rt_start=lines[2].find("time: ")+len("time: ")
    rt_end=lines[2].find(" Statistics")
    rt=time_to_second(lines[2][rt_start:rt_end])
    
    q_start=lines[2].find("Query:")+len("Query:")
    q_end=lines[2].find(", Got")
    queries=int(lines[2][q_start:q_end])
        
    return [wt,rt,queries],True

In [5]:
def get_types(fn):
    s=fn.split("_",3)
    if s[0]=="native":
        return "release", "native","native"
    else:
        return s[1],s[0],s[2]

In [6]:
TOP_REFRESH = 3 #seconds
def get_cpu(csv_fn, routines, wt, rt):
    raw_data = pd.read_csv(csv_fn, delim_whitespace=True)
    cpu = raw_data[(raw_data.loc[:,'VIRT']!='0')].loc[:,'%CPU']
    
    wskip = int((wt/TOP_REFRESH))+1
    rinclude = int(rt/TOP_REFRESH)
    rcpu = cpu[wskip:wskip+rinclude]
    
    wcpu = 100
    rstd = rcpu.std()
    rmin = rcpu.min()
    rmax = rcpu.max()
    rcpu = rcpu.mean()
    return wcpu, rcpu, rmin, rmax,rstd


In [7]:

def calc_efficiency(report):
    report[headers["WriteEfficiency"]] = float(report[headers["Entries"]])/(report[headers["WriteCPU"]]*report[headers["WriteTime"]])
    report[headers["ReadEfficiency"]] = float(report[headers["Queries"]])/(report[headers["QueryCPU"]]*report[headers["QueryTime"]])


In [8]:
headers = {
    "From":"FromFiles",
    "Release":"Release", #Release/Debug
    "OS":"LibOS" , #Native/Gramine/Ego
    "Enclave":"Enclave", #Native/Sim/Sgx
    "Entries":"Entries", #Number
    "Queries":"Queries", #Number
    "QueryRoutines":"QueryRoutines",#Number
    "ServiceRoutines":"ServiceRoutines",#Number
    "WriteTime":"WriteTime",#Seconds
    "WriteCPU":"WriteCPU%" ,#
    "QueryTime":"QueryTime",#Seconds
    "QueryCPU":"QueryCPU%",
    "QueryCPUMin":"QueryCPUMin%",
    "QueryCPUMax":"QueryCPUMax%",
    "Std":"StdevOfQueryCPU",
    "WriteEfficiency":"WriteEfficiency",#Entries/(WriteCPU*WriteTime)
    "ReadEfficiency":"ReadEfficiency",#Queries/(QueryCPU*QueryTime)
}

def generate_single_report(csv_fn, report_fn):
    report = {}    
    [wt,rt,queries],gotit = get_report(report_fn)
    if not gotit:
        return report, gotit
    
    release, enc, libos=get_types(csv_fn)
    
    report[headers["Release"]] =release
    report[headers["Enclave"]] = enc
    report[headers["OS"]] = libos
    
    report[headers["Queries"]]=queries
    
    regex =re.compile(r'\d+')
    (entries, loops, input_routines, services_routines) = tuple((int(x) for x in regex.findall(csv_fn)))
    report[headers["QueryRoutines"]]=input_routines
    report[headers["ServiceRoutines"]]=services_routines
    report[headers["Entries"]]=entries*1000
    
    wcpu,rcpu,rmin, rmax, rstd =get_cpu(csv_fn, (input_routines + services_routines), wt,rt)
    if wcpu==0 or rcpu==0 or pd.isna(rcpu):
        #print("Lack of data to calculate CPU in ", csv_fn,"\n")
        return report, False
    
    report[headers["WriteTime"]]=wt
    report[headers["QueryTime"]]=rt
    report[headers["WriteCPU"]]=wcpu
    report[headers["QueryCPU"]]=rcpu
    report[headers["QueryCPUMin"]]=rmin
    report[headers["QueryCPUMax"]]=rmax
    report[headers["Std"]] = rstd
    
    calc_efficiency(report)
    
    report[headers["From"]]=report_fn + "," + csv_fn
    return report,gotit


In [9]:
def generate_reports():
    nmatch = 0
    csv_num = 0
    report_num = 0
    reports = []
    for csv_fn in os.listdir():
        if not csv_fn.endswith("csv"):
            report_num += 1
            continue
        csv_num += 1
        report_fn = csv_fn.replace(".csv", ".report").replace("cpu","output")
        if not os.path.exists(report_fn):
            nmatch += 1
            continue
        
        report,gotit = generate_single_report(csv_fn,report_fn)
        if not gotit:
            continue
        
        reports.append(report)
    return pd.DataFrame(reports)

In [10]:
os.chdir("C:\\Projects\\sgx\\mini-li\main\\report")
os.getcwd()
df = generate_reports()
df.dropna(how='any')
sort_by=[
    headers["ServiceRoutines"],
    headers["QueryRoutines"],
    headers["Entries"],
    headers["OS"],
    headers["Enclave"],
    headers["Release"]
]
df = df.sort_values(by=sort_by)
df.to_excel("./cpu.xlsx")
df

,Release,Enclave,LibOS,Queries,QueryRoutines,ServiceRoutines,Entries,WriteTime,QueryTime,WriteCPU%,QueryCPU%,QueryCPUMin%,QueryCPUMax%,StdevOfQueryCPU,WriteEfficiency,ReadEfficiency,FromFiles
214,debug,sgx,ego,48000000,0,0,1000,1.000000e-07,71.631568,100,121.386957,94.1,187.5,34.076055,1.000000e+08,5520.326395,"sgx_debug_ego_output_1_6000_0_0.log.report,sgx..."
323,release,sgx,ego,48000000,0,0,1000,1.000000e-07,70.911572,100,113.265217,94.1,156.2,22.393010,1.000000e+08,5976.233520,"sgx_release_ego_output_1_6000_0_0.log.report,s..."
453,release,sim,ego,48000000,0,0,1000,1.000000e-07,61.891627,100,132.643750,94.1,162.5,21.598672,1.000000e+08,5846.858022,"sim_release_ego_output_1_6000_0_0.log.report,s..."
347,release,sgx,gramine,48000000,0,0,1000,5.280000e-04,81.325849,100,148.111111,94.1,694.1,114.083200,1.893939e+04,3984.969407,sgx_release_gramine_output_1_6000_0_0.log.repo...
571,release,sim,gramine,48000000,0,0,1000,6.300000e-04,70.470215,100,149.207692,135.3,162.5,7.910169,1.587302e+04,4565.038319,sim_release_gramine_output_1_6000_0_0.log.repo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,debug,sgx,ego,480000000,5,20,1000000,9.359946e-01,653.008214,100,1219.000000,100.0,1456.0,208.059845,1.068382e+04,603.002188,"sgx_debug_ego_output_1000_60_5_20.log.report,s..."
273,release,sgx,ego,480000000,5,20,1000000,2.419986e+00,830.087178,100,1190.250857,94.1,1506.0,340.656915,4.132255e+03,485.824071,sgx_release_ego_output_1000_60_5_20.log.report...
403,release,sim,ego,480000000,5,20,1000000,9.319946e-01,676.740070,100,1219.917037,93.8,1488.0,208.292427,1.072968e+04,581.418771,sim_release_ego_output_1000_60_5_20.log.report...
521,release,sim,gramine,480000000,5,20,1000000,9.977110e-01,645.687137,100,1435.491935,1018.0,4418.0,722.769397,1.002294e+04,517.867129,sim_release_gramine_output_1000_60_5_20.log.re...
